In [20]:
import pandas as pd
import subprocess
import time
import os

In [21]:
QUANTIDADE_ARQUIVOS = 10

DF = pd.DataFrame({
    "Produto": ["TV", "Playstation", "Celular", "Tablet"],
    "Preco": [2599.99, 4999.00, 1399.99, 1450.00]
})

In [22]:
def gerar_arquivos(quantidade: int = QUANTIDADE_ARQUIVOS) -> None:
    for i in range(quantidade):
        DF.to_csv(f"arquivo_{i}.csv")

def excluir_arquivos():
    subprocess.run("rm -f *.csv", shell=True)

In [23]:
# Gerando arquivos
gerar_arquivos()

In [24]:
# Código original
inicio = time.time()

arquivos = os.listdir()
for arquivo in arquivos:
    if arquivo.endswith(".csv"):
        tabela = pd.read_csv(arquivo)
        total = tabela["Preco"].sum()
        print("Valor total é: ", total)

print(f"FIM EXEC: {time.time() - inicio}")

Valor total é:  10448.98
Valor total é:  10448.98
Valor total é:  10448.98
Valor total é:  10448.98
Valor total é:  10448.98
Valor total é:  10448.98
Valor total é:  10448.98
Valor total é:  10448.98
Valor total é:  10448.98
Valor total é:  10448.98
FIM EXEC: 0.019392013549804688


In [25]:
from joblib import Parallel, delayed

inicio = time.time()

arquivos = os.listdir()

def calcular_preco_total(arquivo: str):
    if arquivo.endswith(".csv"):
        tabela = pd.read_csv(arquivo)
        total = tabela["Preco"].sum()
        # Print nao funciona em parelelo
        # print("Valor total é: ", total)
        return f"Valor total é: {total}"

resultado = Parallel(n_jobs=2)(delayed(calcular_preco_total)(arquivo) for arquivo in arquivos)
print(resultado)
print(f"FIM EXEC: {time.time() - inicio}")

['Valor total é: 10448.98', None, 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98']
FIM EXEC: 3.02856707572937


#### Paralelismo

Para funções rápidas, de alguns segundos não compensa usar multiprocessamento, mas para jobs que levam meia hora, talvez valha

Threading (para tarefas I/O-bound):
Para tarefas que dependem fortemente de operações de entrada e saída (I/O-bound), como leitura de arquivos ou chamadas de rede, o uso de threads pode ser mais eficaz.

In [26]:
from concurrent.futures import ThreadPoolExecutor

inicio = time.time()

def calcular_preco_total(arquivo: str):
    if arquivo.endswith(".csv"):
        tabela = pd.read_csv(arquivo)
        total = tabela["Preco"].sum()
        return f"Valor total é: {total}"

with ThreadPoolExecutor(max_workers=2) as executor:
    resultados = list(executor.map(calcular_preco_total, arquivos))

print(resultados)
print(f"FIM EXEC: {time.time() - inicio}")

['Valor total é: 10448.98', None, 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98', 'Valor total é: 10448.98']
FIM EXEC: 0.028217077255249023


Multiprocessing (para tarefas CPU-bound):
Para tarefas que consomem muito CPU (CPU-bound), como cálculos intensivos, a biblioteca multiprocessing pode ser uma escolha melhor, já que ela cria processos separados que podem usar múltiplos núcleos.

O código abaixo iria funcionar melhor em  um script .py, no jupyter ele não esta conseguindo serializar a função calcular_preco_total

In [27]:


# from multiprocessing import Pool

# inicio = time.time()

# def calcular_preco_total(arquivo: str):
#     if arquivo.endswith(".csv"):
#         tabela = pd.read_csv(arquivo)
#         total = tabela["Preco"].sum()
#         return f"Valor total é: {total}"

# with Pool(processes=2) as pool:
#     resultados = pool.map(calcular_preco_total, arquivos)

# print(resultados)
# print(f"FIM EXEC: {time.time() - inicio}")


In [28]:
## Excluir Arquivos
excluir_arquivos()

### Threading (ThreadPoolExecutor):
Como funciona: Cria múltiplas threads dentro do mesmo processo. Threads compartilham o mesmo espaço de memória e são mais leves que processos, mas como o Python tem o GIL (Global Interpreter Lock), elas não conseguem executar código Python simultaneamente em múltiplos núcleos. No entanto, Threading é muito eficiente para tarefas I/O-bound (como leitura de arquivos ou chamadas de rede), pois essas tarefas não são bloqueadas pelo GIL.
Quando usar: Ideal para operações I/O-bound onde o tempo de espera é maior que o tempo de CPU. Não é tão eficaz para tarefas CPU-bound.

### Multiprocessing (multiprocessing.Pool):
Como funciona: Cria múltiplos processos separados. Cada processo tem seu próprio espaço de memória, o que permite a execução simultânea em múltiplos núcleos de CPU. Como cada processo é isolado, isso evita o GIL e é ideal para tarefas CPU-bound.
Quando usar: Quando você precisa utilizar múltiplos núcleos do CPU para tarefas pesadas (CPU-bound). Também pode ser usado para I/O-bound, mas com overhead maior devido à criação de processos.

### Joblib (Parallel):
Como funciona: Joblib pode usar tanto threads quanto processos. Por padrão, para operações em Python (não nativo), ele usa o backend loky, que cria processos separados. Ele é projetado para tarefas simples e eficazes, com uma interface fácil de usar.
Quando usar: Para paralelizar loops e operações que se beneficiam da execução em múltiplos processos.

### Pathos (ProcessingPool):
Como funciona: Similar ao multiprocessing, Pathos usa processos separados, mas é mais robusto em termos de serialização, usando dill ao invés de pickle. Isso facilita a paralelização em ambientes como Jupyter Notebooks.
Quando usar: Quando você precisa de multiprocessing em ambientes onde o multiprocessing padrão do Python tem problemas, como em Jupyter.

### Dask:
Como funciona: Dask é uma biblioteca mais complexa que pode criar tanto threads quanto processos. Ela também pode ser distribuída, permitindo que você rode tarefas em múltiplos nós de um cluster. Dask usa "gráficos de tarefas" para dividir grandes tarefas em várias menores, que podem ser executadas em paralelo.
Quando usar: Ideal para grandes volumes de dados, tarefas distribuídas, ou quando você precisa de escalabilidade. É uma boa escolha para tarefas que precisam ser executadas tanto em um único computador quanto em clusters.